## Model Setting

- Feature map generator: fine tuning

- Depth cls head

- Depth map head


In [1]:
!nvidia-smi

Thu Mar  9 19:19:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   28C    P8     7W / 320W |   4311MiB / 10018MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install gdown

In [3]:
# import gdown 
# google_path = 'https://drive.google.com/uc?id='
# file_id = '17MHP5y2PwDAnVy05AglJQYotWfqOizpB'
# output_name = 'open.zip'
# gdown.download(google_path+file_id,output_name,quiet=False)

In [4]:
# !unzip open.zip

In [5]:
!pip install timm

In [6]:
import timm
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import albumentations as albu
from albumentations.pytorch import ToTensorV2

import sklearn
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast 

In [7]:
print('numpy verison :', np.__version__)
print('pandas version :', pd.__version__)
print('opencv version :', cv2.__version__)
print('torch version :', torch.__version__)
print('sklearn verison :', sklearn.__version__)
print('albumentations version :', albu.__version__)

numpy verison : 1.19.5
pandas version : 1.1.5
opencv version : 4.5.2
torch version : 1.7.0
sklearn verison : 0.24.2
albumentations version : 1.0.0


In [8]:
CFG = {
    'WIDTH':48,
    'HEIGHT':72,
    'EPOCHS':51,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
    'SEED':41
}

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
simulation_sem_paths = sorted(glob.glob('./data/simulation_data/SEM/*/*/*.png'))

simulation_depth_paths = sorted(glob.glob('./data/simulation_data/Depth/*/*/*.png')*4)

test_sem_path_list = sorted(glob.glob('./data/test/SEM/*.png'))

train_sem_path_list = sorted(glob.glob('./data/train/SEM/*/*/*.png'))

In [11]:
simulation_sem_paths

['./data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0005-01MS_3_itr0.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0005-01MS_3_itr1.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0006-01MS_3_itr0.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0006-01MS_3_itr1.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0009-01MS_3_itr0.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0009-01MS_3_itr1.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0029-01MS_1_itr0.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0029-01MS_1_itr1.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0030-01MS_2_itr0.png',
 './data/simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0030-01MS_2_i

In [12]:
simulation_sem_paths[0].split('/')[-3]

'Case_1'

In [13]:
pd.Series(simulation_sem_paths).apply(lambda x: x.split('/')[-3])

0         Case_1
1         Case_1
2         Case_1
3         Case_1
4         Case_1
           ...  
173299    Case_4
173300    Case_4
173301    Case_4
173302    Case_4
173303    Case_4
Length: 173304, dtype: object

In [14]:
pd.Series(train_sem_path_list).apply(lambda x: x.split('/')[-3])

0        Depth_110
1        Depth_110
2        Depth_110
3        Depth_110
4        Depth_110
           ...    
60659    Depth_140
60660    Depth_140
60661    Depth_140
60662    Depth_140
60663    Depth_140
Length: 60664, dtype: object

In [15]:
label = pd.Series(train_sem_path_list).apply(lambda x: x.split('/')[-3])

In [16]:
label

0        Depth_110
1        Depth_110
2        Depth_110
3        Depth_110
4        Depth_110
           ...    
60659    Depth_140
60660    Depth_140
60661    Depth_140
60662    Depth_140
60663    Depth_140
Length: 60664, dtype: object

In [17]:
# label = pd.concat([pd.Series(train_sem_path_list).apply(lambda x: x.split('/')[-3]),pd.Series(simulation_sem_paths).apply(lambda x: x.split('/')[-3])])

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
encoder = LabelEncoder()

In [20]:
encoder.fit(np.array(label).reshape(-1,1))

LabelEncoder()

In [21]:
onehot = encoder.transform(np.array(label).reshape(-1,1))

In [22]:
onehot

array([0, 0, 0, ..., 3, 3, 3])

In [23]:
timm.list_models('*efficient*')

['efficientformer_l1',
 'efficientformer_l3',
 'efficientformer_l7',
 'efficientnet_b0',
 'efficientnet_b0_g8_gn',
 'efficientnet_b0_g16_evos',
 'efficientnet_b0_gn',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_g8_gn',
 'efficientnet_b3_gn',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',
 'efficientnet_cc_b0_4e',
 'efficientnet_cc_b0_8e',
 'efficientnet_cc_b1_8e',
 'efficientnet_el',
 'efficientnet_el_pruned',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_es_pruned',
 'efficientnet_l2',
 'efficientnet_lite0',
 'efficientnet_lite1',
 'efficientnet_lite2',
 'efficientnet_lite3',
 'efficientnet_lite4',
 'efficientnetv2_l',
 'efficientnetv2_m',
 'efficientnetv2_rw_m',
 'efficientnetv2_rw_s',
 'efficientnetv2_rw_t',
 'efficientnetv2_s',
 'efficientnetv2_xl',
 'gc_efficientnet

In [24]:
timm.list_models('*densenet*')

['densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'tv_densenet121']

In [25]:
timm.list_models('*resnet*')

['cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'eca_resnet33ts',
 'ecaresnet26t',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet50t',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnet200d',
 'ecaresnet269d',
 'ecaresnetlight',
 'ens_adv_inception_resnet_v2',
 'gcresnet33ts',
 'gcresnet50t',
 'gluon_resnet18_v1b',
 'gluon_resnet34_v1b',
 'gluon_resnet50_v1b',
 'gluon_resnet50_v1c',
 'gluon_resnet50_v1d',
 'gluon_resnet50_v1s',
 'gluon_resnet101_v1b',
 'gluon_resnet101_v1c',
 'gluon_resnet101_v1d',
 'gluon_resnet101_v1s',
 'gluon_resnet152_v1b',
 'gluon_resnet152_v1c',
 'gluon_resnet152_v1d',
 'gluon_resnet152_v1s',
 'inception_resnet_v2',
 'lambda_resnet26rpt_256',
 'lambda_resnet26t',
 'lambda_resnet50ts',
 'legacy_seresnet18',
 'legacy_seresnet34',
 'legacy_seresnet50',
 'legacy_seresnet101',
 'legacy_seresnet152',
 'nf_ecaresnet26',
 'nf_ecaresnet50',
 'nf_ecaresnet101',
 'nf_resnet26',
 'nf_resnet50',
 'nf_resnet101',
 'nf_seresnet26',
 'nf_seresnet50',
 'nf_seresnet1

##  Depth class Prediction

- Feature map generator: train

- Depth cls head: train

## ResNet 50

In [26]:
model_name = 'resnet50'

In [27]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.enabled = False
seed_everything(CFG['SEED']) # Seed 고정

In [28]:
class CustomDataset_cls(Dataset):
    def __init__(self, train_sem_path_list, mode):
        self.sem_path_list = train_sem_path_list
        self.mode = mode
        if self.mode == 'train':
            self.transform = albu.Compose([albu.VerticalFlip(),
                                    albu.HorizontalFlip(),
                                    ToTensorV2(),
                                ])

        else:
            
            self.transform = albu.Compose([
                        ToTensorV2(),
                    ])

    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)
        sem_img = sem_img / 255.
        transformed = self.transform(image = sem_img)
        sem_img = transformed['image']

        img_name = sem_path.split('/')[-3]

        if img_name == 'Case_1':
            img_name = 'Depth_110'
        elif img_name == 'Case_2':
            img_name = 'Depth_120'
        elif img_name == 'Case_3':
            img_name = 'Depth_130'
        elif img_name == 'Case_4':
            img_name = 'Depth_140'
#             print(sem_img)
        return sem_img, img_name # B,C,H,W
        
    def __len__(self):
        return len(self.sem_path_list)

In [29]:
path_list = train_sem_path_list + simulation_sem_paths
data_len = len(path_list)
train_idx = np.int_(np.random.choice(data_len, data_len, replace=False))
train_idx

array([ 26255, 208776, 102700, ...,  61324, 132003,  34752])

In [30]:
# array([  7069, 323651, 389351, ..., 323468, 132003, 296896])

In [31]:
train_sem_paths = list(np.array(path_list)[train_idx[:np.int(data_len*0.8)]])
val_sem_paths = list(np.array(path_list)[train_idx[np.int(data_len*0.8):]])
batch_size = CFG['BATCH_SIZE']

train_dataset = CustomDataset_cls(train_sem_paths, mode = 'train')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

val_dataset = CustomDataset_cls(val_sem_paths, mode = 'val')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [32]:
len(train_sem_paths), len(val_sem_paths)

(187174, 46794)

In [33]:
# import torch.optim as optim

outer_model = timm.create_model(model_name, pretrained = True, in_chans=1, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = outer_model.parameters(), lr = CFG["LEARNING_RATE"])
outer_model = outer_model.to(device)
scaler = GradScaler()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [34]:
pbar = tqdm(range(CFG['EPOCHS']))

for epoch in pbar:   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        
        labels = encoder.transform(np.array(labels).reshape(-1,1))
        
        inputs = inputs.to(device).float()
        labels = torch.Tensor(labels).to(device).long()
            

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        with autocast():
            # 순전파 + 역전파 + 최적화를 한 후
            outputs = outer_model(inputs)
            loss = criterion(outputs, labels)
        

        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 통계를 출력합니다.
        running_loss += loss.item()

        
        if i % 500 == 0:    # print every 2000 mini-batches
            pbar.set_description(f'Epochs: {epoch + 1}, {i+1}-th, Train loss: {running_loss / 500:.6f}')
            running_loss = 0.0

    if epoch % 10 == 0:
        correct = 0
        total = 0
        # 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data

                labels = encoder.transform(np.array(labels).reshape(-1,1))

                inputs = inputs.to(device).float()
                labels = torch.Tensor(labels).to(device).long()
                # 신경망에 이미지를 통과시켜 출력을 계산합니다
                outputs = outer_model(inputs)
                outputs = F.softmax(outputs, dim = 1)
                # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
                _, predicted = torch.max(outputs.data, 1)
#                 print(confusion_matrix(labels.cpu().numpy(), predicted.detach().cpu().numpy()))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epochs: {epoch + 1}, Accuracy of the network on the test images: {100 * correct / total} %')
print('Finished Training')

  0%|          | 0/51 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epochs: 1, Accuracy of the network on the test images: 97.30948412189596 %


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0it [00:00, ?it/s]

0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
outer_model.eval()
with torch.no_grad():
    for path in tqdm(test_sem_path_list):
        sem_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
        sem_img = sem_img / 255.
        sem_img = torch.Tensor(sem_img).unsqueeze(0).to(device)
        pred_cls = outer_model(sem_img)
        print(torch.argmax(pred_cls))

  0%|          | 0/25988 [00:00<?, ?it/s]

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

...

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
t

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
t

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
t

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
t

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
torch.save(outer_model.state_dict(), "./weight/outer_model_"+model_name+".pt")        

## resnet34

In [37]:
model_name = 'resnet34'

In [38]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.enabled = False
seed_everything(CFG['SEED']+1) # Seed 고정

In [39]:
class CustomDataset_cls(Dataset):
    def __init__(self, train_sem_path_list, mode):
        self.sem_path_list = train_sem_path_list
        self.mode = mode
        if self.mode == 'train':
            self.transform = albu.Compose([albu.VerticalFlip(),
                                    albu.HorizontalFlip(),
                                    ToTensorV2(),
                                ])

        else:
            
            self.transform = albu.Compose([
                        ToTensorV2(),
                    ])

    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)
        sem_img = sem_img / 255.
        transformed = self.transform(image = sem_img)
        sem_img = transformed['image']

        img_name = sem_path.split('/')[-3]

        if img_name == 'Case_1':
            img_name = 'Depth_110'
        elif img_name == 'Case_2':
            img_name = 'Depth_120'
        elif img_name == 'Case_3':
            img_name = 'Depth_130'
        elif img_name == 'Case_4':
            img_name = 'Depth_140'
#             print(sem_img)
        return sem_img, img_name # B,C,H,W
        
    def __len__(self):
        return len(self.sem_path_list)

In [40]:
path_list = train_sem_path_list + simulation_sem_paths
data_len = len(path_list)
train_idx = np.int_(np.random.choice(data_len, data_len, replace=False))
train_idx

array([124543,  45497, 215669, ..., 131932, 146867, 121958])

In [41]:
# array([157347, 137163, 400206, ..., 131932, 146867, 121958])

In [42]:
train_sem_paths = list(np.array(path_list)[train_idx[:np.int(data_len*0.8)]])
val_sem_paths = list(np.array(path_list)[train_idx[np.int(data_len*0.8):]])
batch_size = CFG['BATCH_SIZE']

train_dataset = CustomDataset_cls(train_sem_paths, mode = 'train')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

val_dataset = CustomDataset_cls(val_sem_paths, mode = 'val')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [43]:
# import torch.optim as optim

outer_model = timm.create_model(model_name, pretrained = True, in_chans=1, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = outer_model.parameters(), lr = CFG["LEARNING_RATE"])
outer_model = outer_model.to(device)
scaler = GradScaler()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34-43635321.pth" to /root/.cache/torch/hub/checkpoints/resnet34-43635321.pth


In [44]:
pbar = tqdm(range(CFG['EPOCHS']))

for epoch in pbar:   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        
        labels = encoder.transform(np.array(labels).reshape(-1,1))
        
        inputs = inputs.to(device).float()
        labels = torch.Tensor(labels).to(device).long()
            

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        with autocast():
            # 순전파 + 역전파 + 최적화를 한 후
            outputs = outer_model(inputs)
            loss = criterion(outputs, labels)
        

        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 통계를 출력합니다.
        running_loss += loss.item()

        
        if i % 500 == 0:    # print every 2000 mini-batches
            pbar.set_description(f'Epochs: {epoch + 1}, {i+1}-th, Train loss: {running_loss / 500:.6f}')
            running_loss = 0.0

    if epoch % 10 == 0:
        correct = 0
        total = 0
        # 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data

                labels = encoder.transform(np.array(labels).reshape(-1,1))

                inputs = inputs.to(device).float()
                labels = torch.Tensor(labels).to(device).long()
                # 신경망에 이미지를 통과시켜 출력을 계산합니다
                outputs = outer_model(inputs)
                outputs = F.softmax(outputs, dim = 1)
                # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
                _, predicted = torch.max(outputs.data, 1)
#                 print(confusion_matrix(labels.cpu().numpy(), predicted.detach().cpu().numpy()))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epochs: {epoch + 1}, Accuracy of the network on the test images: {100 * correct / total} %')
print('Finished Training')

  0%|          | 0/51 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epochs: 1, Accuracy of the network on the test images: 96.39483694490747 %


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0it [00:00, ?it/s]

0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
outer_model.eval()
with torch.no_grad():
    for path in tqdm(test_sem_path_list):
        sem_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
        sem_img = sem_img / 255.
        sem_img = torch.Tensor(sem_img).unsqueeze(0).to(device)
        pred_cls = outer_model(sem_img)
        print(torch.argmax(pred_cls))

  0%|          | 0/25988 [00:00<?, ?it/s]

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

...

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
t

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
t

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
t

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
torch.save(outer_model.state_dict(), "./weight/outer_model_"+model_name+".pt")        

## efficientnet-b3

In [47]:
model_name = 'efficientnet_b3'

In [48]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.enabled = False
seed_everything(CFG['SEED']+2) # Seed 고정

In [49]:
class CustomDataset_cls(Dataset):
    def __init__(self, train_sem_path_list, mode):
        self.sem_path_list = train_sem_path_list
        self.mode = mode
        if self.mode == 'train':
            self.transform = albu.Compose([albu.VerticalFlip(),
                                    albu.HorizontalFlip(),
                                    ToTensorV2(),
                                ])

        else:
            
            self.transform = albu.Compose([
                        ToTensorV2(),
                    ])

    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)
        sem_img = sem_img / 255.
        transformed = self.transform(image = sem_img)
        sem_img = transformed['image']

        img_name = sem_path.split('/')[-3]

        if img_name == 'Case_1':
            img_name = 'Depth_110'
        elif img_name == 'Case_2':
            img_name = 'Depth_120'
        elif img_name == 'Case_3':
            img_name = 'Depth_130'
        elif img_name == 'Case_4':
            img_name = 'Depth_140'
#             print(sem_img)
        return sem_img, img_name # B,C,H,W
        
    def __len__(self):
        return len(self.sem_path_list)

In [50]:
path_list = train_sem_path_list + simulation_sem_paths
data_len = len(path_list)
train_idx = np.int_(np.random.choice(data_len, data_len, replace=False))
train_idx

array([161297, 161097,  66015, ...,  40753, 150848,  14148])

In [51]:
train_sem_paths = list(np.array(path_list)[train_idx[:np.int(data_len*0.8)]])
val_sem_paths = list(np.array(path_list)[train_idx[np.int(data_len*0.8):]])
batch_size = CFG['BATCH_SIZE']

train_dataset = CustomDataset_cls(train_sem_paths, mode = 'train')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

val_dataset = CustomDataset_cls(val_sem_paths, mode = 'val')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [52]:
# import torch.optim as optim

outer_model = timm.create_model(model_name, pretrained = True, in_chans=1, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = outer_model.parameters(), lr = CFG["LEARNING_RATE"])
outer_model = outer_model.to(device)
scaler = GradScaler()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


In [53]:
pbar = tqdm(range(CFG['EPOCHS']))

for epoch in pbar:   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        
        labels = encoder.transform(np.array(labels).reshape(-1,1))
        
        inputs = inputs.to(device).float()
        labels = torch.Tensor(labels).to(device).long()
            

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        with autocast():
            # 순전파 + 역전파 + 최적화를 한 후
            outputs = outer_model(inputs)
            loss = criterion(outputs, labels)
        

        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 통계를 출력합니다.
        running_loss += loss.item()

        
        if i % 500 == 0:    # print every 2000 mini-batches
            pbar.set_description(f'Epochs: {epoch + 1}, {i+1}-th, Train loss: {running_loss / 500:.6f}')
            running_loss = 0.0

    if epoch % 10 == 0:
        correct = 0
        total = 0
        # 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data

                labels = encoder.transform(np.array(labels).reshape(-1,1))

                inputs = inputs.to(device).float()
                labels = torch.Tensor(labels).to(device).long()
                # 신경망에 이미지를 통과시켜 출력을 계산합니다
                outputs = outer_model(inputs)
                outputs = F.softmax(outputs, dim = 1)
                # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
                _, predicted = torch.max(outputs.data, 1)
#                 print(confusion_matrix(labels.cpu().numpy(), predicted.detach().cpu().numpy()))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epochs: {epoch + 1}, Accuracy of the network on the test images: {100 * correct / total} %')
print('Finished Training')

  0%|          | 0/51 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epochs: 1, Accuracy of the network on the test images: 96.80728298499808 %


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epochs: 51, Accuracy of the network on the test images: 99.41231781852375 %
Finished Training


In [54]:
outer_model.eval()
with torch.no_grad():
    for path in tqdm(test_sem_path_list):
        sem_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
        sem_img = sem_img / 255.
        sem_img = torch.Tensor(sem_img).unsqueeze(0).to(device)
        pred_cls = outer_model(sem_img)
        print(torch.argmax(pred_cls))

  0%|          | 0/25988 [00:00<?, ?it/s]

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

...

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(2, device='cuda:0')
t

In [55]:
torch.save(outer_model.state_dict(), "./weight/outer_model_"+model_name+".pt")        

## densenet169

In [56]:
model_name = 'densenet169'

In [57]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.enabled = False
seed_everything(CFG['SEED']+3) # Seed 고정

In [58]:
class CustomDataset_cls(Dataset):
    def __init__(self, train_sem_path_list, mode):
        self.sem_path_list = train_sem_path_list
        self.mode = mode
        if self.mode == 'train':
            self.transform = albu.Compose([albu.VerticalFlip(),
                                    albu.HorizontalFlip(),
                                    ToTensorV2(),
                                ])

        else:
            
            self.transform = albu.Compose([
                        ToTensorV2(),
                    ])

    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)
        sem_img = sem_img / 255.
        transformed = self.transform(image = sem_img)
        sem_img = transformed['image']

        img_name = sem_path.split('/')[-3]

        if img_name == 'Case_1':
            img_name = 'Depth_110'
        elif img_name == 'Case_2':
            img_name = 'Depth_120'
        elif img_name == 'Case_3':
            img_name = 'Depth_130'
        elif img_name == 'Case_4':
            img_name = 'Depth_140'
#             print(sem_img)
        return sem_img, img_name # B,C,H,W
        
    def __len__(self):
        return len(self.sem_path_list)

In [59]:
path_list = train_sem_path_list + simulation_sem_paths
data_len = len(path_list)
train_idx = np.int_(np.random.choice(data_len, data_len, replace=False))
train_idx

array([116556,  82652,  96245, ..., 156845, 200099,  14100])

In [60]:
train_sem_paths = list(np.array(path_list)[train_idx[:np.int(data_len*0.8)]])
val_sem_paths = list(np.array(path_list)[train_idx[np.int(data_len*0.8):]])
batch_size = CFG['BATCH_SIZE']

train_dataset = CustomDataset_cls(train_sem_paths, mode = 'train')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

val_dataset = CustomDataset_cls(val_sem_paths, mode = 'val')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [61]:
# import torch.optim as optim

outer_model = timm.create_model(model_name, pretrained = True, in_chans=1, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = outer_model.parameters(), lr = CFG["LEARNING_RATE"])
outer_model = outer_model.to(device)
scaler = GradScaler()

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


In [62]:
pbar = tqdm(range(CFG['EPOCHS']))

for epoch in pbar:   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        
        labels = encoder.transform(np.array(labels).reshape(-1,1))
        
        inputs = inputs.to(device).float()
        labels = torch.Tensor(labels).to(device).long()
            

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        with autocast():
            # 순전파 + 역전파 + 최적화를 한 후
            outputs = outer_model(inputs)
            loss = criterion(outputs, labels)
        

        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 통계를 출력합니다.
        running_loss += loss.item()

        
        if i % 500 == 0:    # print every 2000 mini-batches
            pbar.set_description(f'Epochs: {epoch + 1}, {i+1}-th, Train loss: {running_loss / 500:.6f}')
            running_loss = 0.0

    if epoch % 10 == 0:
        correct = 0
        total = 0
        # 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data

                labels = encoder.transform(np.array(labels).reshape(-1,1))

                inputs = inputs.to(device).float()
                labels = torch.Tensor(labels).to(device).long()
                # 신경망에 이미지를 통과시켜 출력을 계산합니다
                outputs = outer_model(inputs)
                outputs = F.softmax(outputs, dim = 1)
                # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
                _, predicted = torch.max(outputs.data, 1)
#                 print(confusion_matrix(labels.cpu().numpy(), predicted.detach().cpu().numpy()))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epochs: {epoch + 1}, Accuracy of the network on the test images: {100 * correct / total} %')
print('Finished Training')

  0%|          | 0/51 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epochs: 1, Accuracy of the network on the test images: 96.84574945505834 %


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

Epochs: 41, Accuracy of the network on the test images: 99.35034406120442 %


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [63]:
outer_model.eval()
with torch.no_grad():
    for path in tqdm(test_sem_path_list):
        sem_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
        sem_img = sem_img / 255.
        sem_img = torch.Tensor(sem_img).unsqueeze(0).to(device)
        pred_cls = outer_model(sem_img)
        print(torch.argmax(pred_cls))

  0%|          | 0/25988 [00:00<?, ?it/s]

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
t

...

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
t

In [64]:
torch.save(outer_model.state_dict(), "./weight/outer_model_"+model_name+".pt")        